In [19]:
# 0. Load libraries #

import numpy as np
import pandas as pd
import os, time
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import svm
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()

In [29]:
# 1. Load data #

time1 = time.time()

path = '../input/titanic/train.csv'
df = pd.read_csv(path) 
df.drop(columns=['Name', 'Ticket', 'Cabin'],inplace=True)
pred=pd.read_csv('../input/titanic/test.csv')
pred.drop(columns=['Name', 'Ticket', 'Cabin'],inplace=True)

print(df.shape, pred.shape)
df.head()

(891, 9) (418, 8)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [30]:
# 2. pEDA #

df.Survived.value_counts()


0    549
1    342
Name: Survived, dtype: int64

In [31]:
# 3. Train-test split #

df.drop(columns = ['PassengerId'], inplace = True)
train_y = df[['Survived']]
train_x = df.drop(columns = ['Survived'])

bin_cols = [col for col in train_x.columns if train_x[col].nunique()==2]
cat_cols = [col for col in train_x.columns if train_x[col].nunique() in range(3,10)]
num_cols = list(set(train_x.columns)-set(cat_cols)-set(bin_cols))

print('binary features: ', bin_cols, 'categorical features: ', cat_cols, 'numerical features: ', num_cols)

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape)

binary features:  ['Sex'] categorical features:  ['Pclass', 'SibSp', 'Parch', 'Embarked'] numerical features:  ['Age', 'Fare']
(712, 7) (179, 7) (712, 1)


In [32]:
# 4. Misisng values #

X_train[num_cols] = X_train[num_cols].fillna(value='median')
X_test[num_cols] = X_train[num_cols].fillna(value='median')
X_train[cat_cols] = X_train[cat_cols].fillna(value='mode')
X_test[cat_cols] = X_train[cat_cols].fillna(value='mode')
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 301 to 37
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       712 non-null    object 
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  712 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 60.7+ KB


In [33]:
X_train0 = X_train.copy()
X_train0

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
301,3,male,median,2,0,23.2500,Q
309,1,female,30.0,0,0,56.9292,C
516,2,female,34.0,0,0,10.5000,S
120,2,male,21.0,2,0,73.5000,S
570,2,male,62.0,0,0,10.5000,S
...,...,...,...,...,...,...,...
715,3,male,19.0,0,0,7.6500,S
767,3,female,30.5,0,0,7.7500,Q
72,2,male,21.0,0,0,73.5000,S
235,3,female,median,0,0,7.5500,S


In [34]:
# 5. Feature engineering #

feature_transformer = ColumnTransformer([
("num", StandardScaler(), num_cols),
("cat", OneHotEncoder(), cat_cols),
("bin", LabelBinarizer, bin_cols)])

feature_transformer.fit_transform(X_train)


ValueError: could not convert string to float: 'median'

In [26]:
ohc = OneHotEncoder()
ohc.fit_transform(X_train[cat_cols])

<712x21 sparse matrix of type '<class 'numpy.float64'>'
	with 2848 stored elements in Compressed Sparse Row format>